# Lab 4.1: Data Preprocessing - Aligning CORINE with Sentinel-2

**Date:** 19 February 2026  
**Instructor:** Samy (online)

## ⏱️ Time Allocation (2h 20min)
- **Part 1 (20 min):** Understanding CORINE Land Cover data
- **Part 2 (15 min):** Setting up the processing environment
- **Part 3 (45 min):** Stacking Sentinel-2 bands
- **Part 4 (45 min):** Aligning CORINE to S2 tile geometry
- **Part 5 (15 min):** Verification and visualization

## 🎯 Learning Objectives

### Core (Essential - Everyone Should Complete)
- ✅ Understand CORINE Land Cover classification system
- ✅ Stack Sentinel-2 bands into multi-band GeoTIFFs using rasterio
- ✅ Reproject and align CORINE data to match S2 tile geometry
- ✅ Verify alignment and data quality

### Optional (For Early Finishers)
- 🔵 Process multiple S2 tiles in batch
- 🔵 Explore different resampling methods

---

## Section 1: Introduction to CORINE Land Cover

### What is CORINE?
**CORINE** (Coordination of Information on the Environment) Land Cover is a European programme providing consistent land cover/land use information across Europe.

### Key Features:
- 🗺️ **Coverage:** All EU member states + some neighboring countries
- 📐 **Resolution:** 100m (raster version)
- 🏷️ **Classes:** 44 land cover classes in 3-level hierarchy
- 📅 **Updates:** 1990, 2000, 2006, 2012, 2018 (we use 2018)

### Class Hierarchy (simplified):
```
Level 1: Artificial surfaces, Agricultural areas, Forest, Wetlands, Water bodies
Level 2: Urban fabric, Industrial areas, Arable land, Forests, etc.
Level 3: 44 detailed classes (what we use)
```

### ⚠️ Important Note
CORINE does **NOT** cover Iceland! If you're working with Icelandic tiles, you'll need alternative land cover data (e.g., ESA WorldCover).

### Why Alignment is Needed
- **CORINE CRS:** EPSG:3035 (ETRS89-LAEA Europe)
- **Sentinel-2 CRS:** UTM zones (e.g., EPSG:32633 for zone 33N)
- **CORINE Resolution:** 100m
- **Sentinel-2 Resolution:** 10m (for bands B02, B03, B04, B08)

We must reproject CORINE to match each S2 tile's coordinate system and extent.

## Section 2: Environment Setup

### Import Required Libraries

We use **rasterio** for all raster operations - it's a Pythonic wrapper around GDAL that works reliably in our HPC environment.

In [ ]:
import os
from pathlib import Path
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.enums import Resampling

# Set matplotlib config directory to avoid permission issues
os.environ['MPLCONFIGDIR'] = f"/tmp/matplotlib_{os.environ.get('USER', 'user')}"

print("✓ Libraries imported successfully")
print(f"  rasterio version: {rasterio.__version__}")
print(f"  NumPy version: {np.__version__}")

### Define CORINE Class Mapping

The CORINE raster uses simplified codes 1-44 (not the full CLC codes like 111, 112, etc.).

In [ ]:
# CORINE class descriptions (simplified codes 1-44 used in raster)
CORINE_CLASSES = {
    # Artificial surfaces (1-11)
    1: "Continuous urban fabric",
    2: "Discontinuous urban fabric",
    3: "Industrial or commercial units",
    4: "Road and rail networks",
    5: "Port areas",
    6: "Airports",
    7: "Mineral extraction sites",
    8: "Dump sites",
    9: "Construction sites",
    10: "Green urban areas",
    11: "Sport and leisure facilities",
    
    # Agricultural areas (12-22)
    12: "Non-irrigated arable land",
    13: "Permanently irrigated land",
    14: "Rice fields",
    15: "Vineyards",
    16: "Fruit trees and berry plantations",
    17: "Olive groves",
    18: "Pastures",
    19: "Annual crops with permanent crops",
    20: "Complex cultivation patterns",
    21: "Agriculture with natural vegetation",
    22: "Agro-forestry areas",
    
    # Forest and semi-natural areas (23-34)
    23: "Broad-leaved forest",
    24: "Coniferous forest",
    25: "Mixed forest",
    26: "Natural grasslands",
    27: "Moors and heathland",
    28: "Sclerophyllous vegetation",
    29: "Transitional woodland-shrub",
    30: "Beaches, dunes, sands",
    31: "Bare rocks",
    32: "Sparsely vegetated areas",
    33: "Burnt areas",
    34: "Glaciers and perpetual snow",
    
    # Wetlands (35-39)
    35: "Inland marshes",
    36: "Peat bogs",
    37: "Salt marshes",
    38: "Salines",
    39: "Intertidal flats",
    
    # Water bodies (40-44)
    40: "Water courses",
    41: "Water bodies",
    42: "Coastal lagoons",
    43: "Estuaries",
    44: "Sea and ocean",
    
    48: "No data"
}

# Color mapping for visualization
CORINE_COLORS = {
    1: '#E6004D', 2: '#FF0000', 3: '#CC4DF2', 4: '#CC0000', 5: '#E6CCCC',
    6: '#E6CCE6', 7: '#A600CC', 8: '#A64DCC', 9: '#FF4DFF', 10: '#FFA6FF',
    11: '#FFE6FF', 12: '#FFFFA8', 13: '#FFFF00', 14: '#E6E600', 15: '#E68000',
    16: '#F2A64D', 17: '#E6A600', 18: '#E6E64D', 19: '#FFE6A6', 20: '#FFE64D',
    21: '#E6CC4D', 22: '#F2CCA6', 23: '#80FF00', 24: '#00A600', 25: '#4DFF00',
    26: '#CCF24D', 27: '#A6FF80', 28: '#A6E64D', 29: '#A6F200', 30: '#E6E6E6',
    31: '#CCCCCC', 32: '#CCFFCC', 33: '#000000', 34: '#A6E6CC', 35: '#A6A6FF',
    36: '#4D4DFF', 37: '#CCCCFF', 38: '#E6E6FF', 39: '#A6A6E6', 40: '#00CCF2',
    41: '#80F2E6', 42: '#00FFA6', 43: '#A6FFE6', 44: '#E6F2FF'
}

print(f"✓ Defined {len(CORINE_CLASSES)} CORINE land cover classes")

## Section 3: Configure Processing Parameters

### 🔧 Set Your Paths and Options

**Edit the cell below to configure:**
1. Which Sentinel-2 tiles to process
2. Output directory for aligned data

In [ ]:
# ============================================================
# CONFIGURATION - Edit these values!
# ============================================================

# Get username for default paths
USER = os.environ.get('USER', 'user')

# --- Path Configuration ---
# Directory containing your downloaded Sentinel-2 .SAFE folders
S2_DATA_DIR = f"/p/scratch/training2600/{USER}/data"

# CORINE Land Cover raster (shared location)
CORINE_PATH = "/p/scratch/training2600/CORINE/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif"

# Output directory for processed data (stacked bands + aligned CORINE)
OUTPUT_DIR = f"/p/scratch/training2600/{USER}/data/aligned_data"

# --- Tile Selection ---
# Options:
#   'all'  - Process all .SAFE directories found in S2_DATA_DIR
#   'list' - Process only tiles listed in TILE_LIST below
TILE_SELECTION = 'all'

# If TILE_SELECTION = 'list', specify which tiles to process:
TILE_LIST = [
    # Add your specific tile names here, e.g.:
    # "S2A_MSIL2A_20181019T102031_N0500_R065_T33UUP_20230813T105225.SAFE",
]

# --- Processing Options ---
SKIP_EXISTING = True  # Skip tiles that already have output files

# ============================================================
print("Configuration:")
print(f"  S2 Data Directory: {S2_DATA_DIR}")
print(f"  CORINE Path: {CORINE_PATH}")
print(f"  Output Directory: {OUTPUT_DIR}")
print(f"  Tile Selection: {TILE_SELECTION}")

### Discover Available Tiles

In [ ]:
def find_s2_tiles(data_dir, selection='all', tile_list=None):
    """
    Find Sentinel-2 .SAFE directories to process.
    
    Parameters:
    -----------
    data_dir : str
        Directory containing .SAFE folders
    selection : str
        'all' to find all tiles, 'list' to use tile_list
    tile_list : list
        List of specific tile names to process
    
    Returns:
    --------
    list : List of Path objects for .SAFE directories
    """
    data_path = Path(data_dir)
    
    if not data_path.exists():
        print(f"❌ Data directory not found: {data_dir}")
        return []
    
    if selection == 'all':
        tiles = sorted(data_path.glob("*.SAFE"))
    else:
        tiles = []
        for tile_name in (tile_list or []):
            tile_path = data_path / tile_name
            if tile_path.exists():
                tiles.append(tile_path)
            else:
                print(f"⚠ Tile not found: {tile_name}")
    
    return tiles

# Find tiles
available_tiles = find_s2_tiles(S2_DATA_DIR, TILE_SELECTION, TILE_LIST)

print(f"\n📂 Found {len(available_tiles)} Sentinel-2 tile(s) to process:")
for i, tile in enumerate(available_tiles, 1):
    print(f"  {i}. {tile.name}")

if len(available_tiles) == 0:
    print("\n⚠ No tiles found! Check your S2_DATA_DIR path.")

### Verify CORINE Data

In [ ]:
# Check CORINE file exists and get basic info
corine_path = Path(CORINE_PATH)

if not corine_path.exists():
    print(f"❌ CORINE file not found: {CORINE_PATH}")
    print("\nPlease check the path or download CORINE data.")
else:
    with rasterio.open(corine_path) as src:
        print("✓ CORINE Land Cover 2018")
        print(f"  File: {corine_path.name}")
        print(f"  Size: {src.width} x {src.height} pixels")
        print(f"  Resolution: {src.res[0]}m x {src.res[1]}m")
        print(f"  CRS: {src.crs}")
        print(f"  NoData value: {src.nodata}")
        print(f"  Bounds: {src.bounds}")

## Section 4: Processing Functions

### Stack Sentinel-2 Bands

Combine the 10m resolution bands (B02, B03, B04, B08) into a single multi-band GeoTIFF using rasterio.

In [ ]:
def stack_s2_bands(safe_path, output_path):
    """
    Stack Sentinel-2 bands (B02, B03, B04, B08) into a single GeoTIFF.
    
    Parameters:
    -----------
    safe_path : Path
        Path to .SAFE directory
    output_path : Path
        Output path for stacked GeoTIFF
    
    Returns:
    --------
    dict : Info about the stacked file (crs, bounds, etc.) or None on failure
    """
    bands = ['B02', 'B03', 'B04', 'B08']  # Blue, Green, Red, NIR
    resolution = '10m'
    
    try:
        # Find band files
        band_paths = {}
        for band_name in bands:
            pattern = f"**/R{resolution}/*_{band_name}_{resolution}.jp2"
            matches = list(safe_path.glob(pattern))
            if matches:
                band_paths[band_name] = matches[0]
            else:
                print(f"  ⚠ {band_name} not found")
                return None
        
        # Read first band for metadata
        with rasterio.open(band_paths['B02']) as src:
            profile = src.profile.copy()
            height, width = src.height, src.width
            crs = src.crs
            transform = src.transform
            bounds = src.bounds
        
        # Update profile for output
        profile.update(
            driver='GTiff',
            count=len(bands),
            dtype='uint16',
            compress='lzw',
            tiled=True,
            blockxsize=256,
            blockysize=256
        )
        
        # Create output and write bands
        with rasterio.open(output_path, 'w', **profile) as dst:
            for i, (band_name, band_path) in enumerate(band_paths.items(), start=1):
                with rasterio.open(band_path) as src:
                    data = src.read(1)
                    dst.write(data, i)
                    dst.set_band_description(i, band_name)
        
        return {
            'crs': crs,
            'width': width,
            'height': height,
            'transform': transform,
            'bounds': bounds,
            'bands': bands
        }
        
    except Exception as e:
        print(f"  ❌ Band stacking failed: {e}")
        import traceback
        traceback.print_exc()
        return None


print("✓ Band stacking function defined")

### Align CORINE to Sentinel-2

Reproject CORINE from EPSG:3035 to match the S2 tile's UTM projection and extent using rasterio's warp functionality.

In [ ]:
def align_corine_to_s2(corine_path, s2_path, output_path):
    """
    Align CORINE raster to match Sentinel-2 tile geometry using rasterio.
    
    Parameters:
    -----------
    corine_path : Path
        Path to CORINE GeoTIFF
    s2_path : Path
        Path to stacked S2 GeoTIFF (reference)
    output_path : Path
        Output path for aligned CORINE
    
    Returns:
    --------
    bool : Success status
    """
    try:
        # Get S2 geometry (target)
        with rasterio.open(s2_path) as s2_src:
            dst_crs = s2_src.crs
            dst_transform = s2_src.transform
            dst_width = s2_src.width
            dst_height = s2_src.height
            dst_bounds = s2_src.bounds
        
        print(f"    Target CRS: {dst_crs}")
        print(f"    Target size: {dst_width} x {dst_height} pixels")
        print(f"    Target bounds: {dst_bounds}")
        
        # Open CORINE and reproject
        with rasterio.open(corine_path) as src:
            # Create output profile
            out_profile = src.profile.copy()
            out_profile.update(
                driver='GTiff',
                crs=dst_crs,
                transform=dst_transform,
                width=dst_width,
                height=dst_height,
                compress='lzw',
                tiled=True
            )
            
            # Create output raster
            with rasterio.open(output_path, 'w', **out_profile) as dst:
                # Reproject using nearest neighbor (for categorical data)
                reproject(
                    source=rasterio.band(src, 1),
                    destination=rasterio.band(dst, 1),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
        
        return True
        
    except Exception as e:
        print(f"  ❌ CORINE alignment failed: {e}")
        import traceback
        traceback.print_exc()
        return False


def check_corine_coverage(corine_path):
    """
    Check if aligned CORINE has valid data.
    
    Returns:
    --------
    tuple : (has_valid_data, unique_classes, coverage_percent)
    """
    with rasterio.open(corine_path) as src:
        data = src.read(1)
    
    total_pixels = data.size
    valid_mask = (data >= 1) & (data <= 44)
    valid_pixels = np.sum(valid_mask)
    coverage_percent = (valid_pixels / total_pixels) * 100
    
    unique = np.unique(data[valid_mask])
    
    return len(unique) > 0, unique, coverage_percent


print("✓ CORINE alignment functions defined")

## Section 5: Process Tiles

### Main Processing Loop

This cell will process all selected tiles:
1. Stack S2 bands into multi-band GeoTIFF
2. Reproject and align CORINE to match S2 geometry
3. Verify alignment and report coverage

In [ ]:
def process_tile_alignment(safe_path, corine_path, output_dir, skip_existing=True):
    """
    Process a single Sentinel-2 tile: stack bands and align CORINE.
    
    Returns:
    --------
    dict : Processing results
    """
    tile_name = safe_path.stem
    result = {
        'tile': tile_name,
        'status': 'unknown',
        's2_stacked': None,
        'corine_aligned': None,
        'crs': None,
        'n_classes': 0,
        'coverage_percent': 0
    }
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Output paths
    s2_stacked = output_dir / f"{tile_name}_stacked.tif"
    corine_aligned = output_dir / f"corine_aligned_{tile_name}.tif"
    
    # Check for existing output
    if skip_existing and s2_stacked.exists() and corine_aligned.exists():
        print(f"  ⏭ Skipping (outputs exist)")
        has_coverage, classes, coverage = check_corine_coverage(corine_aligned)
        result['status'] = 'skipped'
        result['s2_stacked'] = str(s2_stacked)
        result['corine_aligned'] = str(corine_aligned)
        result['n_classes'] = len(classes)
        result['coverage_percent'] = coverage
        return result
    
    # Step 1: Stack bands
    print(f"  📦 Stacking S2 bands...")
    stack_info = stack_s2_bands(safe_path, s2_stacked)
    if stack_info is None:
        result['status'] = 'failed_stacking'
        return result
    
    result['crs'] = str(stack_info['crs'])
    print(f"    Created: {s2_stacked.name}")
    print(f"    Size: {stack_info['width']} x {stack_info['height']} pixels")
    print(f"    CRS: {stack_info['crs']}")
    
    # Step 2: Align CORINE
    print(f"  🗺️ Aligning CORINE to S2 geometry...")
    if not align_corine_to_s2(corine_path, s2_stacked, corine_aligned):
        result['status'] = 'failed_alignment'
        return result
    
    # Step 3: Check coverage
    print(f"  🔍 Checking CORINE coverage...")
    has_coverage, classes, coverage = check_corine_coverage(corine_aligned)
    
    if not has_coverage:
        print(f"  ⚠ No valid CORINE data (tile outside coverage?)")
        result['status'] = 'no_coverage'
        return result
    
    result['status'] = 'success'
    result['s2_stacked'] = str(s2_stacked)
    result['corine_aligned'] = str(corine_aligned)
    result['n_classes'] = len(classes)
    result['coverage_percent'] = coverage
    
    print(f"    CORINE classes found: {len(classes)}")
    print(f"    Valid coverage: {coverage:.1f}%")
    
    return result


print("✓ Tile processing function defined")

In [ ]:
# ============================================================
# MAIN PROCESSING LOOP
# ============================================================

print("="*70)
print("ALIGNING CORINE WITH SENTINEL-2 TILES")
print("="*70)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Tiles to process: {len(available_tiles)}")
print(f"Output directory: {OUTPUT_DIR}")
print()

# Store results
all_results = []

# Process each tile
for i, tile_path in enumerate(available_tiles, 1):
    print(f"\n[{i}/{len(available_tiles)}] Processing: {tile_path.name}")
    
    result = process_tile_alignment(
        safe_path=tile_path,
        corine_path=CORINE_PATH,
        output_dir=OUTPUT_DIR,
        skip_existing=SKIP_EXISTING
    )
    
    all_results.append(result)
    
    if result['status'] == 'success':
        print(f"  ✅ Success: {result['n_classes']} classes, {result['coverage_percent']:.1f}% coverage")
    elif result['status'] == 'skipped':
        print(f"  ⏭ Skipped (already processed)")
    else:
        print(f"  ❌ Status: {result['status']}")

# Summary
print("\n" + "="*70)
print("ALIGNMENT SUMMARY")
print("="*70)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

successful = [r for r in all_results if r['status'] == 'success']
skipped = [r for r in all_results if r['status'] == 'skipped']
failed = [r for r in all_results if r['status'] not in ['success', 'skipped']]

print(f"\nResults:")
print(f"  ✅ Successful: {len(successful)}")
print(f"  ⏭ Skipped: {len(skipped)}")
print(f"  ❌ Failed: {len(failed)}")
print(f"\n  Output directory: {OUTPUT_DIR}")

## Section 6: Verify Alignment

### Visualize Alignment Results

In [ ]:
def visualize_alignment(s2_path, corine_path, title="Alignment Verification", save_path=None):
    """
    Visualize the S2 image and aligned CORINE side by side.
    
    Parameters:
    -----------
    s2_path : str or Path
        Path to stacked S2 GeoTIFF
    corine_path : str or Path
        Path to aligned CORINE GeoTIFF
    title : str
        Plot title
    save_path : str or Path, optional
        Path to save the figure (PNG format)
    """
    # Load S2 data
    with rasterio.open(s2_path) as src:
        # Read RGB bands (B04, B03, B02 = bands 3, 2, 1)
        r = src.read(3).astype(float)  # B04 (Red)
        g = src.read(2).astype(float)  # B03 (Green)
        b = src.read(1).astype(float)  # B02 (Blue)
    
    # Create RGB composite
    rgb = np.stack([r, g, b], axis=-1)
    p2, p98 = np.percentile(rgb, (2, 98))
    rgb = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)
    
    # Load CORINE data
    with rasterio.open(corine_path) as src:
        corine = src.read(1)
    
    # Create CORINE colormap
    from matplotlib.colors import ListedColormap
    unique_classes = np.unique(corine[(corine >= 1) & (corine <= 44)])
    colors = [CORINE_COLORS.get(c, '#888888') for c in range(45)]
    cmap = ListedColormap(colors)
    
    # Create figure (subsample for display)
    step = 10
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # S2 RGB
    axes[0].imshow(rgb[::step, ::step])
    axes[0].set_title('Sentinel-2 RGB (B04, B03, B02)', fontsize=12)
    axes[0].axis('off')
    
    # CORINE
    axes[1].imshow(corine[::step, ::step], cmap=cmap, vmin=0, vmax=44)
    axes[1].set_title('Aligned CORINE Land Cover', fontsize=12)
    axes[1].axis('off')
    
    # Overlay
    axes[2].imshow(rgb[::step, ::step])
    corine_masked = np.ma.masked_where((corine < 1) | (corine > 44), corine)
    axes[2].imshow(corine_masked[::step, ::step], cmap=cmap, vmin=0, vmax=44, alpha=0.4)
    axes[2].set_title('S2 + CORINE Overlay', fontsize=12)
    axes[2].axis('off')
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    # Save figure if path provided
    if save_path:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Figure saved to: {save_path}")
    
    plt.show()
    
    # Print class distribution
    print("\nCORINE Classes in this tile:")
    for c in sorted(unique_classes):
        count = np.sum(corine == c)
        pct = count / corine.size * 100
        print(f"  Class {c:2d}: {CORINE_CLASSES.get(c, 'Unknown')[:30]:30s} - {count:,} pixels ({pct:.1f}%)")


# Results directory for saving figures
RESULTS_DIR = f"/p/project1/training2600/{USER}/results"

# Visualize first successful result
successful_results = [r for r in all_results if r['status'] in ['success', 'skipped']]

if successful_results:
    result = successful_results[0]
    tile_name = result['tile']
    
    # Create save path for the visualization
    save_path = Path(RESULTS_DIR) / f"alignment_{tile_name[:50]}.png"
    
    print(f"Visualizing: {tile_name[:50]}...")
    visualize_alignment(
        result['s2_stacked'], 
        result['corine_aligned'],
        title=f"Alignment: {tile_name[:40]}...",
        save_path=save_path
    )
else:
    print("No successful results to visualize.")

### View Processing Results Summary

In [ ]:
# Display detailed results
print("Detailed Results per Tile:")
print("-" * 90)
print(f"{'Tile':<50} {'Status':<15} {'Classes':<10} {'Coverage':<10}")
print("-" * 90)

for result in all_results:
    tile_short = result['tile'][:48] + ".." if len(result['tile']) > 50 else result['tile']
    coverage = f"{result['coverage_percent']:.1f}%" if result['coverage_percent'] > 0 else "-"
    classes = str(result['n_classes']) if result['n_classes'] > 0 else "-"
    print(f"{tile_short:<50} {result['status']:<15} {classes:<10} {coverage:<10}")

print("-" * 90)

# List output files
print("\n📁 Output Files:")
output_path = Path(OUTPUT_DIR)
if output_path.exists():
    for f in sorted(output_path.glob("*.tif")):
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  {f.name} ({size_mb:.1f} MB)")
else:
    print(f"  Output directory not found: {OUTPUT_DIR}")

## Summary & Next Steps

### What We Accomplished
✅ Stacked Sentinel-2 bands (B02, B03, B04, B08) into multi-band GeoTIFFs  
✅ Reprojected CORINE from EPSG:3035 to match S2 tile UTM projection  
✅ Aligned CORINE extent and resolution to match S2 imagery  
✅ Verified alignment and coverage  

### Output Files
- `*_stacked.tif` - Stacked S2 bands (B02=Blue, B03=Green, B04=Red, B08=NIR)
- `corine_aligned_*.tif` - CORINE reprojected and aligned to each S2 tile

### Key Takeaways
- CORINE uses EPSG:3035 (LAEA Europe) - must reproject to S2's UTM zone
- Nearest-neighbor resampling preserves categorical class values
- Some tiles may have partial or no CORINE coverage (edges of Europe)
- Alignment is essential for accurate pixel-wise label extraction

### Next Lab: Patch Extraction & Normalization
In **Lab 4.2** (5 March), we'll extract training patches from the aligned data:
- Load stacked S2 bands and aligned CORINE
- Extract fixed-size patches with center-pixel labels
- Apply normalization to the data
- Handle class imbalance
- Create ML-ready training datasets

---

**Great work!** Your data is now aligned and ready for patch extraction! 🚀